# Utility Methods

In [1]:
#| default_exp utils

In [2]:
#| export
import CodeSyntaxConcept
import random
import numpy as np

In [3]:
#| export
# From: https://github.com/github/CodeSearchNet/tree/master/function_parser
def traverse(
    node,       # tree-sitter node
    results,    # list to append results to
) -> None:
    """Traverse in a recursive way, a tree-sitter node and append results to a list."""
    if node.type == 'string':
        results.append(node)
        return
    for n in node.children:
        traverse(n, results)
    if not node.children:
        results.append(node)

In [4]:
#| export 
def find_nodes(
    node,               #Tree sitter ast tree
    target_node_type,   #Target node type to search in the tree
    results,            #List to append the resutls to
) -> None: 
    """Traverses the tree and find the specified node type"""
    if node.type == target_node_type:
        results.append(node)
        return
    for n in node.children:
        find_nodes(n, target_node_type, results)

In [ ]:
#| export 
def find_parent_nodes(
    node,               #Tree sitter ast tree
    results,            #List to append the resutls to
) -> None: 
    """Traverses the tree and find the parent nodes"""
    if node.children:
        results.append(node)
        for n in node.children:
            find_parent_nodes(n, results)

In [1]:
#| export
def unroll_node_types(
    nested_node_types: dict  # node_types from tree-sitter
) -> list: # list of node types
    def iterate_and_unroll_dict(nested_node_types: dict, all_node_types: set):
        for key, value in nested_node_types.items():
            if key == 'type' and type(value) == str:
                all_node_types.add(value)
            if type(value) == dict:
                iterate_and_unroll_dict(value, all_node_types)
            if type(value) == list:
                for element in value:
                    iterate_and_unroll_dict(element, all_node_types) 
    all_node_types = set()
    for dictionary in nested_node_types:
        iterate_and_unroll_dict(dictionary, all_node_types)
    all_node_types.add('ERROR')
    return list(all_node_types)

In [6]:
#| export
def convert_to_offset(
    point,              #point to convert
    lines: list         #list of lines in the source code
    ):
        """Convert the point to an offset"""
        row, column = point
        chars_in_rows = sum(map(len, lines[:row])) + row
        chars_in_columns = len(lines[row][:column])
        offset = chars_in_rows + chars_in_columns
        return offset

In [1]:
#| export
def get_test_sets(test_set, language, max_token_number, model_tokenizer, with_ranks=False, num_proc=1):
    subset = test_set.filter(lambda sample: True if sample['language']== language 
            and len(sample['func_code_tokens']) < max_token_number
            and len(model_tokenizer.tokenizer(sample['whole_func_string'])['input_ids']) <= max_token_number
            else False, num_proc=num_proc)
    return subset

In [1]:
#| export
def get_sub_set_test_set(test_set, test_size:int):
    sub_samples = []
    for sample in test_set:
        sub_samples.append(sample)
        if len(sub_samples)>=test_size:
            break
    return sub_samples

In [ ]:
#| export
def get_random_sub_set_test_set(test_set, test_size:int):
    sub_samples = []
    while len(sub_samples)<test_size:
        random_index = random.randrange(0,len(test_set))
        sub_samples.append(test_set[random_index])
    return sub_samples

In [ ]:
#| export
def bootstrapping( np_data, np_func, size ):
    """Create a bootstrap sample given data and a function
    For instance, a bootstrap sample of means, or mediands. 
    The bootstrap replicates are a long as the original size
    we can choose any observation more than once (resampling with replacement:np.random.choice)
    """
    
    #Cleaning NaNs
    #np_data_clean = np_data[ np.logical_not( np.isnan(np_data) ) ] 
    
    #The size of the bootstrap replicate is as big as size
    #Creating the boostrap replicates as long as the orignal data size
    #This strategy might work as imputation 
    bootstrap_repl = [ np_func( np.random.choice( np_data, size=len(np_data) ) ) for i in range( size ) ]
    
    #logging.info("Covariate: " + cov) #Empirical Mean
    #logging.info("Empirical Mean: " + str(np.mean(np_data_clean))) #Empirical Mean
    #logging.info("Bootstrapped Mean: " + str( np.mean(bootstrap_repl) ) ) #Bootstrapped Mean
    
    return np.array( bootstrap_repl )